In [8]:
import pandas as pd
from itertools import product
from typing import Iterable
pd.options.display.max_columns = 100


def generate_test_vectors(n_inputs:int=0,labels:list[str]=[]):
    # labels[0] is MSB label
    if n_inputs:
        return pd.DataFrame(
            [*product(*((False, True) for _ in range(n_inputs)))],
            columns=[chr(65 + _) for _ in reversed(range(n_inputs))],
        )
    return pd.DataFrame(
        [*product(*((False, True) for _ in range(len(labels))))],
        columns=labels,
    )

def validate_checks(df: pd.DataFrame, checks: dict[str, pd.Series], out_col: str = "Q"):
    for var, check in checks.items():
        if out_col not in df.columns:
            raise ValueError(f"Output column {out_col} not in DataFrame")
        tmp = df[var].where(check)
        xored = df[out_col][tmp.notna()] ^ df[var][tmp.notna()]
        if xored.any() and not xored.all():
            raise ValueError(f"Mismatch in {var} check: {xored}")

def calc_cases(df: pd.DataFrame, checks: dict[str, pd.Series]):
    out = pd.DataFrame(index=df.index)
    for var, check in checks.items():
        tmp = df[var].where(check)
        out[f"{var}/gnd"] = tmp.where(tmp == True)
        out[f"{var}/vdd"] = tmp.where(tmp == False)
    out[out.notna()] = 1
    return out.fillna(0).T


def validate_SaF_vectors(saf_cases:pd.DataFrame, test_vectors:Iterable[int]):
    if not saf_cases.iloc[:,test_vectors].any(axis=1).all():
        raise ValueError(f"Test vectors {test_vectors} do not cover all checks")
    return test_vectors

saf_test_vectors = dict()
"""Stores test vectors for SaF testing for a part"""
IDDQ_faults = dict()
"""Stores faults detected by IDDQ testing for a part"""
# to select test cases:
# - select column with most ones and add it to test vector,
# - repeat, but skip rows, where testing is covered
# - if testing vestor is full, stop

'Stores faults detected by IDDQ testing for a part'

In [9]:
# C17 circuit

df = generate_test_vectors(5)

# define/calculate intermediate signals in the structure
df["n1"] = ~(df.A & df.C)
df["n2"] = ~(df.D & df.C)
df["n3"] = ~(df.n2 & df.B)
df["n4"] = ~(df.n2 & df.E)

# define/calculate outputs
df["Y1"] = ~(df.n1 & df.n3)
df["Y2"] = ~(df.n3 & df.n4)

# define/calculate passthrough masks for signals
checks = {
    "A": (df.C & df.n3),
    "B": (df.n2 & (df.n1 | df.n4)),
    "C": (df.A & df.n3)
    | (df.D & df.B & (df.n1 | df.n4 | df.E))
    | (df.D & df.E & df.n3),
    "D": (df.C & df.B & (df.n1 | df.n4 | df.E)) | (df.C & df.E & df.n3),
    "E": (df.n2 & df.n3),
    "n1": df.n3,
    "n2": (df.B & (df.n1 | df.n4)) | (df.E & df.n3) | (df.B & df.E),
    "n3": df.n1 | df.n4,
    "n4": df.n3,
    "Y1": df.index >= 0,
    "Y2": df.index >= 0,
}

# calculate test vectors coverage for a SaF model
out_c17 = calc_cases(df, checks)
saf_test_vectors["C17"] = validate_SaF_vectors(out_c17, [4, 6, 25, 31])

In [10]:
out_c17

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
A/gnd,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1
A/vdd,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
B/gnd,0,0,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0
B/vdd,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0
C/gnd,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1
C/vdd,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0
D/gnd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1
D/vdd,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0
E/gnd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0
E/vdd,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
# NO42 circuit

df = generate_test_vectors(4)

df["n3"] = ~(df.A | df.B)
df["n1"] = ~(df.D | df.C)
df["n2"] = ~(df.n1 & df.n3)

df["Q"] = ~(df.n2)

checks = {
    "A": (~df.B & df.n1),
    "B": (~df.A & df.n1),
    "C": (~df.D & df.n3),
    "D": (~df.C & df.n3),
    "n1": df.n3,
    "n2": df.index >= 0,
    "n3": df.n1,
    "Q": df.index >= 0,
}

validate_checks(df, checks)

out_no42 = calc_cases(df, checks)
saf_test_vectors["NO42"] = validate_SaF_vectors(out_no42, [0,1,2,4,8])

In [12]:
out_no42

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
A/gnd,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A/vdd,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
B/gnd,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
B/vdd,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C/gnd,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
C/vdd,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
D/gnd,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
D/vdd,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
n1/gnd,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
n1/vdd,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0


In [13]:
docs_faults_files = {
    "C17": "C17.csv",
    "NO42": "NO42.csv",
}

coverages = pd.DataFrame(index=['all', 'SaF', 'all+IDDQ', 'SaF+IDDQ'], columns=docs_faults_files.keys(),dtype=float)
for part, f_path in docs_faults_files.items():
    faults = pd.read_csv(f_path, sep=";", index_col=0)
    # coverage vectors:
    def calc_coverage(coverage_vector):
        return coverage_vector.any(axis=1).mean()
    coverages.at['all',part] = calc_coverage(faults==1)
    coverages.at['SaF',part] = calc_coverage(faults.iloc[:, saf_test_vectors[part]]==1)
    if part in IDDQ_faults:
        faults.loc[IDDQ_faults[part],0]=1    
    coverages.at['all+IDDQ',part]  = calc_coverage(faults==1)
    coverages.at['SaF+IDDQ',part]  = calc_coverage(faults.iloc[:, saf_test_vectors[part]]==1)
coverages

,C17,NO42
all,0.813953,0.977273
SaF,0.773256,0.977273
all+IDDQ,0.813953,0.977273
SaF+IDDQ,0.773256,0.977273


In [14]:
# print("Stopień pokrycia wszystkich możliwych błędów przez wektory testowe:")
# print()
# print("C17")
# print(f"- wyznaczone dla modelu SaF: {c17_saf_coverage:.2f}%")
# print(f"- obejmujące wszystkie kombinacje stanów portów wejściowych: {c17_all_coverage:.2f}%")
# print()
# print("NO42")
# print(f"- wyznaczone dla modelu SaF: {no42_saf_coverage:.2f}%")
# print(f"- obejmujące wszystkie kombinacje stanów portów wejściowych: {no42_all_coverage:.2f}%")